In [134]:
import pandas as pd
import numpy as np
import math

In [135]:
from chardet.universaldetector import UniversalDetector
detector = UniversalDetector()
with open('cities.csv', 'rb') as fh:
    for line in fh:
        detector.feed(line)
        if detector.done:
            break
    detector.close()
print(detector.result)

{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}


In [136]:
cities = pd.read_csv('cities.csv', encoding = 'utf-8')
cities.sample(5)

,Индекс,Тип региона,Регион,Тип района,Район,Тип города,Город,Тип н/п,Н/п,Код КЛАДР,...,Уровень по ФИАС,Признак центра района или региона,Код ОКАТО,Код ОКТМО,Код ИФНС,Часовой пояс,Широта,Долгота,Федеральный округ,Население
250,238730.0,обл,Калининградская,р-н,Краснознаменский,г,Краснознаменск,NaN,NaN,3900700100000,...,4: город,1,27218501000,2.771300e+10,3914,UTC+2,54.945261,22.492851,Северо-Западный,3522
154,162600.0,обл,Вологодская,NaN,NaN,г,Череповец,NaN,NaN,3500000200000,...,4: город,0,19430000000,1.973000e+07,3528,UTC+3,59.126921,37.909224,Северо-Западный,312311
520,102321.0,обл,Московская,NaN,NaN,г,Домодедово,NaN,NaN,5000000100000,...,4: город,0,46409000000,4.670900e+07,5000,UTC+3,55.436390,37.766626,Центральный,96123
759,391300.0,обл,Рязанская,NaN,NaN,г,Касимов,NaN,NaN,6200000400000,...,4: город,1,61405000000,6.170500e+10,6226,UTC+3,54.937379,41.391291,Центральный,33494
103,601300.0,обл,Владимирская,р-н,Камешковский,г,Камешково,NaN,NaN,3300600100000,...,4: город,1,17225501000,1.762510e+10,3340,UTC+3,56.348871,40.995568,Центральный,13113


In [137]:
cities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1112 entries, 0 to 1111
Data columns (total 21 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Индекс                             1108 non-null   float64
 1   Тип региона                        1112 non-null   object 
 2   Регион                             1112 non-null   object 
 3   Тип района                         629 non-null    object 
 4   Район                              629 non-null    object 
 5   Тип города                         1108 non-null   object 
 6   Город                              1108 non-null   object 
 7   Тип н/п                            2 non-null      object 
 8   Н/п                                2 non-null      object 
 9   Код КЛАДР                          1112 non-null   int64  
 10  Код ФИАС                           1112 non-null   object 
 11  Уровень по ФИАС                    1112 non-null   objec

In [138]:
cities.columns

Index(['Индекс', 'Тип региона', 'Регион', 'Тип района', 'Район', 'Тип города',
       'Город', 'Тип н/п', 'Н/п', 'Код КЛАДР', 'Код ФИАС', 'Уровень по ФИАС',
       'Признак центра района или региона', 'Код ОКАТО', 'Код ОКТМО',
       'Код ИФНС', 'Часовой пояс', 'Широта', 'Долгота', 'Федеральный округ',
       'Население'],
      dtype='object')

In [139]:
cities.drop(['Индекс', 'Тип региона', 'Тип н/п', 'Н/п', 'Код КЛАДР', 'Код ФИАС', 'Уровень по ФИАС',
       'Признак центра района или региона', 'Код ОКАТО', 'Код ОКТМО',
       'Код ИФНС', 'Федеральный округ'], axis = 1, inplace = True)

In [140]:
cities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1112 entries, 0 to 1111
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Регион        1112 non-null   object 
 1   Тип района    629 non-null    object 
 2   Район         629 non-null    object 
 3   Тип города    1108 non-null   object 
 4   Город         1108 non-null   object 
 5   Часовой пояс  1112 non-null   object 
 6   Широта        1112 non-null   float64
 7   Долгота       1112 non-null   float64
 8   Население     1112 non-null   object 
dtypes: float64(2), object(7)
memory usage: 78.3+ KB


In [141]:
cities[cities['Город'] == 'Казань']

,Регион,Тип района,Район,Тип города,Город,Часовой пояс,Широта,Долгота,Население
924,Татарстан,NaN,NaN,г,Казань,UTC+3,55.794388,49.111531,1216965


In [142]:
cities['Часовой пояс'] = cities['Часовой пояс'].str.replace('UTC', '') #удаляю UTC из часового пояса

In [143]:
cities['Часовой пояс'] = cities['Часовой пояс'].apply(lambda x: int(x))

In [144]:
int(cities[cities['Город'] == 'Казань']['Часовой пояс'].values)+2.5

5.5

In [191]:
#Отсекаем часовые пояса +- 2 часа
near_Kazan = cities[cities['Часовой пояс'].between(float(cities[cities['Город'] == 'Самара']['Часовой пояс'].values)-2.5,
                                            float(cities[cities['Город'] == 'Самара']['Часовой пояс'].values)+2.5,
                                            inclusive = True)]

In [192]:
# Отсекаем города по Долготе
near_Kazan = near_Kazan[near_Kazan['Долгота'].between(float(near_Kazan[near_Kazan['Город'] == 'Самара']['Долгота'].values)-1,
                                            float(near_Kazan[near_Kazan['Город'] == 'Самара']['Долгота'].values)+1,
                                            inclusive = True)]

In [205]:
math.cos(round(float(near_Kazan[near_Kazan['Город'] == 'Самара']['Широта'].values), 6) * 0.017)

0.618221918105964

In [193]:
max_latitude = 40000/360 * math.cos(float(near_Kazan[near_Kazan['Город'] == 'Самара']['Широта'].values))
print(max_latitude)

-108.62567738421929


In [189]:
latitude = round(111.111/(max_latitude/60)/60, 6)
latitude

-1.02288

In [187]:
near_Kazan = near_Kazan[near_Kazan['Широта'].between(float(near_Kazan[near_Kazan['Город'] == 'Самара']
                                                           ['Широта'].values)-latitude,
                                                     float(near_Kazan[near_Kazan['Город'] == 'Самара']
                                                           ['Широта'].values)+latitude,
                                            inclusive = True)]

In [188]:
near_Kazan

,Регион,Тип района,Район,Тип города,Город,Часовой пояс,Широта,Долгота,Население


In [165]:
near_sity_s = dict()

near_sity_s['Казань'] = list()

for i in near_Kazan['Город']:
    near_sity_s['Казань'].append(i)

print(near_sity_s) 

{'Казань': []}
{'Казань': ['Уржум', 'Волжск', 'Арск', 'Болгар', 'Буинск', 'Зеленодольск', 'Иннополис', 'Казань', 'Лаишево', 'Тетюши', 'Козловка']}


In [206]:
def near_city(city_name):
    near_sity_s = dict() #создаю словарь, что бы добавить города
    near_sity_s[city_name] = list()
    
    #Отсекаем часовые пояса +- 2 часа
    near_Kazan = cities[cities['Часовой пояс'].between(float(cities[cities['Город'] == city_name]['Часовой пояс'].values)-2.5,
                                            float(cities[cities['Город'] == city_name ]['Часовой пояс'].values)+2.5,
                                            inclusive = True)]
    
    # Отсекаем города по Долготе
    near_Kazan = cities[cities['Долгота'].between(float(near_Kazan[near_Kazan['Город'] == city_name]['Долгота'].values)-1,
                                            float(near_Kazan[near_Kazan['Город'] == city_name]['Долгота'].values)+1,
                                            inclusive = True)]
    
    latitude = 40000/360 * math.cos(float(near_Kazan[near_Kazan['Город'] == city_name]['Широта'].values)*0.017)
    latitude = round(111.111/(latitude/60)/60, 6)
    
    # Отсекаем города по Широте
    near_Kazan = near_Kazan[near_Kazan['Широта'].between(float(near_Kazan[near_Kazan['Город'] == city_name]
                                                           ['Широта'].values)-latitude,
                                                     float(near_Kazan[near_Kazan['Город'] == city_name]
                                                           ['Широта'].values)+latitude,
                                            inclusive = True)]
    
    for i in near_Kazan['Город']:
        near_sity_s[city_name].append(i)

    return near_sity_s

Отсекаем часовые пояса +\- 2 часа
Считам длину долготы - разница +\- 111 км
Длина 1 градуса широты 40000\*cos(широты) - 1 градус - длина широты \ 360, в градусе 60 минут - 
   пересчитываем в десятичные доли - 1 градус + 1 мин\60
Находим по разницы широты +\- 1 градус, долготы расчётная разница, список городов
Выводим списко городов

In [211]:
near_city('Сызрань')

{'Сызрань': ['Октябрьск',
  'Сызрань',
  'Тольятти',
  'Балаково',
  'Пугачев',
  'Хвалынск',
  'Новоульяновск',
  'Сенгилей',
  'Ульяновск']}